In [ ]:
import pandas as pd
import csv
import requests
from contextlib import closing
import numpy as np

In [ ]:
import datetime
import dateutil
from datetime import datetime, timedelta, time, date

In [ ]:
citibike_2001 = pd.read_csv('https://s3.amazonaws.com/tripdata/202001-citibike-tripdata.csv.zip')

In [ ]:
citibike_2002 = pd.read_csv('https://s3.amazonaws.com/tripdata/202002-citibike-tripdata.csv.zip')
citibike_2003 = pd.read_csv('https://s3.amazonaws.com/tripdata/202003-citibike-tripdata.csv.zip')
citibike_2004 = pd.read_csv('https://s3.amazonaws.com/tripdata/202004-citibike-tripdata.csv.zip')

In [ ]:
citibike_20 = pd.concat([citibike_2001,citibike_2002,citibike_2003,citibike_2004])

In [ ]:
citibike_20.shape

(4138645, 15)

In [ ]:
citibike_20.tail()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
682757,1680,2020-04-30 23:59:00.7780,2020-05-01 00:27:01.7600,447,8 Ave & W 52 St,40.763707,-73.985162,2021,W 45 St & 8 Ave,40.759291,-73.988597,43888,Subscriber,1989,1
682758,1334,2020-04-30 23:59:23.6460,2020-05-01 00:21:38.2190,3292,5 Ave & E 93 St,40.785785,-73.957481,509,9 Ave & W 22 St,40.745497,-74.001971,36853,Subscriber,1973,1
682759,1334,2020-04-30 23:59:33.6760,2020-05-01 00:21:48.2150,3292,5 Ave & E 93 St,40.785785,-73.957481,509,9 Ave & W 22 St,40.745497,-74.001971,37808,Subscriber,1981,1
682760,234,2020-04-30 23:59:56.9720,2020-05-01 00:03:51.9600,469,Broadway & W 53 St,40.763441,-73.982681,495,W 47 St & 10 Ave,40.762699,-73.993012,20635,Subscriber,1996,2
682761,267,2020-04-30 23:59:59.3510,2020-05-01 00:04:26.8110,3812,University Pl & E 14 St,40.734814,-73.992085,503,E 20 St & Park Ave,40.738274,-73.987520,37530,Subscriber,1980,1


In [ ]:
citibike_20_clean = citibike_20[citibike_20['tripduration'] <= 24*60*60*1] #remove trip over 1 day and less than 1 min
citibike_20_clean= citibike_20_clean[citibike_20_clean['tripduration'] >= 60]

In [ ]:
citibike_20_clean.shape

(4137191, 15)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
citibike_taxizone = pd.read_csv("drive/My Drive/sjoined_citibike.csv", index_col=0)

In [ ]:
citibike_taxizone.head()

,stationName,AvailableDocks,totalDocks,latitude,longitude,StatusValue,StatusKey,availableBikes,Address,lastCommunicationTime,State,geometry,index_right,shape_area,objectid,shape_leng,location_id,zone,borough,venue_counts
Id,,,,,,,,,,,,,,,,,,,,
281,Grand Army Plaza & Central Park S,47,66,40.764397,-73.973715,In Service,1,19,Grand Army Plaza & Central Park S,11/18/19 14:21,NY,POINT (-73.97371465000001 40.7643971),162,0.000041,163,0.034177,163,Midtown North,Manhattan,5.0
468,Broadway & W 56 St,11,65,40.765265,-73.981923,In Service,1,53,Broadway & W 56 St,11/18/19 14:23,NY,POINT (-73.98192338 40.7652654),162,0.000041,163,0.034177,163,Midtown North,Manhattan,5.0
3457,E 58 St & Madison Ave,0,46,40.763026,-73.972095,In Service,1,45,E 58 St & Madison Ave,11/18/19 14:24,NY,POINT (-73.97209525 40.76302594),162,0.000041,163,0.034177,163,Midtown North,Manhattan,5.0
3724,7 Ave & Central Park South,4,39,40.766741,-73.979069,In Service,1,35,7 Ave & Central Park South,11/18/19 14:24,NY,POINT (-73.97906899 40.76674056),162,0.000041,163,0.034177,163,Midtown North,Manhattan,5.0
3809,W 55 St & 6 Ave,9,53,40.763189,-73.978434,In Service,1,40,W 55 St & 6 Ave,11/18/19 14:24,NY,POINT (-73.97843399999999 40.763189),162,0.000041,163,0.034177,163,Midtown North,Manhattan,5.0


In [ ]:
citibike_taxizone.shape

(860, 20)

In [ ]:
citibike_dic ={key:citibike_taxizone['location_id'][key] for key in citibike_taxizone.index}

In [ ]:
citibike_20_clean['start_station_tx']=citibike_20_clean['start station id'].map(citibike_dic)

In [ ]:
citibike_20_clean['end_station_tx']=citibike_20_clean['end station id'].map(citibike_dic)

In [ ]:
citibike_20_clean.shape

(4137191, 17)

In [ ]:
citibike_20_clean.to_csv('citibike_20_clean.csv')

In [ ]:
with open('drive/My Drive/citibike_20_clean.csv','r')as fi:
    reader = csv.DictReader(fi)
    for record in reader:
      print(record)
      break

OrderedDict([('', '0'), ('tripduration', '789'), ('starttime', '2020-01-01 00:00:55.3900'), ('stoptime', '2020-01-01 00:14:05.1470'), ('start station id', '504'), ('start station name', '1 Ave & E 16 St'), ('start station latitude', '40.73221853'), ('start station longitude', '-73.98165557'), ('end station id', '307'), ('end station name', 'Canal St & Rutgers St'), ('end station latitude', '40.71427487'), ('end station longitude', '-73.98990025'), ('bikeid', '30326'), ('usertype', 'Subscriber'), ('birth year', '1992'), ('gender', '1'), ('start_station_tx', '224.0'), ('end_station_tx', '232.0')])


In [ ]:
taxizone_citi_inflow_20 = {}
taxizone_citi_outflow_20 = {}
taxizone_citi_loop_20 = {}

with open('drive/My Drive/citibike_20_clean.csv','r')as fi:
    reader = csv.DictReader(fi)
    for record in reader:
        date_hour_do = record['stoptime'][:13]
        date_hour_pu = record['starttime'][:13]
        dropoff_ID = record['end_station_tx'][:-2]
        pickup_ID = record['start_station_tx'][:-2]

        if dropoff_ID != '' and pickup_ID != '': 
          if dropoff_ID == pickup_ID:
            taxizone_citi_loop_20[(date_hour_do, dropoff_ID)] = taxizone_citi_loop_20.get((date_hour_do, dropoff_ID),0)+1
        
          elif dropoff_ID != pickup_ID:
            taxizone_citi_inflow_20[(date_hour_do, dropoff_ID)] = taxizone_citi_inflow_20.get((date_hour_do, dropoff_ID),0)+1
            taxizone_citi_outflow_20[(date_hour_pu, pickup_ID)] = taxizone_citi_outflow_20.get((date_hour_pu, pickup_ID),0)+1

In [ ]:
import datetime

dt = datetime.datetime(2020, 1, 1)
end = datetime.datetime(2020, 5, 1, 1, 0, 0)
step = datetime.timedelta(hours=1)

result = []
fulldate = []

while dt < end:
    result.append(dt.strftime('%Y-%m-%d %H:%M:%S'))
    fulldate.append(dt.strftime('%m/%d/%Y %H:%M:%S'))
    dt += step

In [ ]:
taxizoneID = list(range(1,264))

In [ ]:
keys = [(x,str(y)) for x in result for y in taxizoneID]

In [ ]:
citi20 = pd.DataFrame({'map' : keys})

In [ ]:
citi20['map2'] = [(x[:13],y) for (x,y) in citi20['map']]

In [ ]:
citi20['inflow_counts'] = citi20.map2.map(taxizone_citi_inflow_20)
citi20['outflow_counts'] = citi20.map2.map(taxizone_citi_outflow_20)
citi20['loop_counts'] = citi20.map2.map(taxizone_citi_loop_20)

In [ ]:
citi20 = citi20.fillna(0)

In [ ]:
citi20['time'] = [x for (x,y) in citi20['map']]
citi20['taxizoneID'] = [int(y) for (x,y) in citi20['map']]

In [ ]:
citi20['time'] = [datetime.strptime(x, '%Y-%m-%d %H:%M:%S') for x in citi20['time']]

In [ ]:
citi20['hour'] = [x.hour for x in citi20['time']]
citi20['weekday'] = citi20[['time']].apply(lambda x: datetime.strftime(x['time'], '%A'), axis=1)
citi20['week'] = [x.isocalendar()[1] for x in citi20['time']]

In [ ]:
def timebin(hour):
    if hour == 20:
        return 1
    if hour == 21:
        return 1
    if hour == 22:
        return 1
    if hour == 23:
        return 1
    if hour == 0:
        return 2
    if hour == 1:
        return 2
    if hour == 2:
        return 2
    if hour == 3:
        return 2
    else:
        return 0
    
hourgroups = {1:'8pm - 12am', 
              2: '12am - 4am', 
              0:'daytime'}

In [ ]:
citi20['timegroup'] = citi20['hour'].apply(timebin)
citi20['timegroupstr'] = citi20['timegroup'].map(hourgroups)

In [ ]:
citi20

,map,map2,inflow_counts,outflow_counts,loop_counts,time,taxizoneID,hour,weekday,week,timegroup,timegroupstr
0,"(2020-01-01 00:00:00, 1)","(2020-01-01 00, 1)",0.0,0.0,0.0,2020-01-01,1,0,Wednesday,1,2,12am - 4am
1,"(2020-01-01 00:00:00, 2)","(2020-01-01 00, 2)",0.0,0.0,0.0,2020-01-01,2,0,Wednesday,1,2,12am - 4am
2,"(2020-01-01 00:00:00, 3)","(2020-01-01 00, 3)",0.0,0.0,0.0,2020-01-01,3,0,Wednesday,1,2,12am - 4am
3,"(2020-01-01 00:00:00, 4)","(2020-01-01 00, 4)",5.0,1.0,0.0,2020-01-01,4,0,Wednesday,1,2,12am - 4am
4,"(2020-01-01 00:00:00, 5)","(2020-01-01 00, 5)",0.0,0.0,0.0,2020-01-01,5,0,Wednesday,1,2,12am - 4am
...,...,...,...,...,...,...,...,...,...,...,...,...
764010,"(2020-05-01 00:00:00, 259)","(2020-05-01 00, 259)",0.0,0.0,0.0,2020-05-01,259,0,Friday,18,2,12am - 4am
764011,"(2020-05-01 00:00:00, 260)","(2020-05-01 00, 260)",0.0,0.0,0.0,2020-05-01,260,0,Friday,18,2,12am - 4am
764012,"(2020-05-01 00:00:00, 261)","(2020-05-01 00, 261)",0.0,0.0,0.0,2020-05-01,261,0,Friday,18,2,12am - 4am
764013,"(2020-05-01 00:00:00, 262)","(2020-05-01 00, 262)",0.0,0.0,0.0,2020-05-01,262,0,Friday,18,2,12am - 4am


In [ ]:
from google.colab import files
citi20.to_csv('citi20.csv') 
files.download('citi20.csv')